In [25]:
import numpy as np
import math
T=5
NX=4
NU=2
DT=0.2
WB=110
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        print(curr_state.v,a,DT,next_v)
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)
    
    def _get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER
            
        A, B, C = get_linear_model_matrix(curr_state.v, curr_state.yaw, delta)
        
        X=np.array([curr_state.x,curr_state.y,curr_state.v,curr_state.yaw],dtype=np.float64)
        U=np.array([a,delta],dtype=np.float64)
        [next_x,next_y,next_v,next_yaw]=np.matmul(A,X) + np.matmul(B,U)+ C
        
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw),[A,B,C]


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion._get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


    
def get_linear_model_matrix(v, phi, delta):
        A = np.zeros((NX, NX))
        A[0, 0] = 1.0
        A[1, 1] = 1.0
        A[2, 2] = 1.0
        A[3, 3] = 1.0
        A[0, 2] = DT * math.cos(phi)
        A[0, 3] = - DT * v * math.sin(phi)#
        A[1, 2] = DT * math.sin(phi)
        A[1, 3] = DT * v * math.cos(phi)#
        A[3, 2] = DT * math.tan(delta) / WB

        B = np.zeros((NX, NU))
        B[2, 0] = DT
        B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

        C = np.zeros(NX)
        C[0] = DT * v * math.sin(phi) * phi
        C[1] = - DT * v * math.cos(phi) * phi
        C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
        #C = np.zeros(NX)
        return A, B, C
    
def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
def calc_cost( x, u,xref,t,_B):
        
        R = np.diag([0.01, 0.01])  # input cost matrix
        Rd = np.diag([0.01, 1.0])  # input difference cost matrix
        Q = np.diag([1.0, 1.0, 0.5, 50])  # state cost matrix
        Qf = Q  # state final matrix
        

        #cost += cvxpy.quad_form(u[:, t], R)
        print(u,R)
        l = np_quad_form(u, R) + np_quad_form(xref - x, Q)
        
        
        #if t < (T - 1):
        #    l = np_quad_form(u[:, t + 1] - u[:, t], Rd)
            
        
        #l = np_quad_form(xref[:, T] - x[:, T], Qf)


        # compute derivatives of cost
        
        l_x1=2*Q[0,0]*(xref[0]-x[0])*(-1)
        l_x2=2*Q[1,1]*(xref[1]-x[1])*(-1)
        l_x3=2*Q[2,2]*(xref[2]-x[2])*(-1)
        l_x4=2*Q[3,3]*(xref[3]-x[3])*(-1)
            
        l_x=np.array([l_x1,l_x2,l_x3,l_x4])
        l_xx=2 * Q
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2], Q[3,3]],dtype=np.float64)
        error=np.array(xref-x)
        #x=Ax+Bu+C
        #dx/du=d(B*u)=B
        l_u1 = 2. * u[0] * R[0,0] + np.sum(2*Qdiag*error*(-1)*_B[:,0])
        l_u2 = 2. * u[1] * R[1,1] + np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        l_u = np.array([l_u1, l_u2])

        l_uu = 2. * R
        
        l_ux = np.zeros((NU, NX))

        return l, l_x, l_xx, l_u, l_uu, l_ux

In [26]:
def sim_n_steps(x0,U,xref):
    X=np.zeros((NX,T+1),dtype=np.float64)
    X[:,0]=x0
    _MS=Motion()
    for t in range(T):
        _state=State(x=X[0,t],y=X[1,t],v=X[2,t],yaw=X[3,t])
        _MS.set_state(_state)
        nst,Const=_MS._get_next_state(U[0,t],U[1,t])
        B=Const[1]
        x_t1=[nst.x,nst.y,nst.v,nst.yaw]
        u_t=U[:,t]
        xref_t=xref[:,t]
        cc=calc_cost(x_t1,u_t,xref_t,t,B)
        X[:,t+1]=x_t1.copy()
        print(cc[0])
    
    return X,cc

Test=True
if Test==True:
    _xref=np.ones((4,5+1),dtype=np.float64)
    _xref[0,:]=5
    _xref[1,:]=5
    _xref[2,:]=5
    _xref[3,:]=5
    _dref=np.zeros((5),dtype=np.float64)

    _x0=np.zeros((NX),dtype=np.float64)
    _x0[2]=1.0
    _u=np.zeros((NU,T),dtype=np.float64)
    _MS=Motion()
    ret=sim_n_steps(_x0,_u,_xref)
    print(ret[0],'\n\n',ret[1][0],'\n',ret[1][3])

[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
1306.04
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
1304.16
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
1302.36
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
1300.64
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
1299.0
[[0.  0.2 0.4 0.6 0.8 1. ]
 [0.  0.  0.  0.  0.  0. ]
 [1.  1.  1.  1.  1.  1. ]
 [0.  0.  0.  0.  0.  0. ]] 

 1299.0 
 [-0.8        -0.90909091]


In [27]:
import numpy as np
from copy import copy, deepcopy



class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=50
        self.dt=0.2
        self.lamb_factor=10.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.01
    def ilqr(self, x0, U,xref): 
        #U = np.zeros((NX,T),dtype=np.float64) if U is None else U
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                # simulate forward using the current control trajectory
                X, cost = sim_n_steps(x0, U,xref)
                oldcost = copy(cost) # copy for exit condition check

                # 
                f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
                f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
                # for storing quadratized cost function 

                l = np.zeros((self.tN,1)) # immediate state cost 
                l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
                l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
                l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
                l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
                l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
                # for everything except final state
                for t in range(self.tN):
                    # x(t+1) = f(x(t), u(t)) = x(t) + dx(t) * dt
                    # linearized dx(t) = np.dot(A(t), x(t)) + np.dot(B(t), u(t))
                    # f_x = (np.eye + A(t)) * dt
                    # f_u = (B(t)) * dt
                    # continuous --> discrete
                    A, B, C = get_linear_model_matrix(X[2,t],X[3,t], U[1,t])
                    f_x[t] = A
                    f_u[t] = B 

                    
                    (l[t], l_x[t], l_xx[t], l_u[t], l_uu[t], l_ux[t]) = calc_cost(X[:,t], U[:,t],xref[:,t],t,B)
                    

                # and for final state
                #l[-1], l_x[-1], l_xx[-1] = self.cost_final(X[-1])

                sim_new_trajectory = False

            V = l[-1].copy() # value function
            V_x = l_x[-1].copy() # dV / dx
            V_xx = l_xx[-1].copy() # d^2 V / dx^2
            k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
            K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

            # work backwards to solve for V, Q, k, and K
            for t in range(self.tN-2, -1, -1):
                
                Q_x = l_x[t] + np.dot(f_x[t].T, V_x) 
                Q_u = l_u[t] + np.dot(f_u[t].T, V_x)

                Q_xx = l_xx[t] + np.dot(f_x[t].T, np.dot(V_xx, f_x[t])) 
                Q_ux = l_ux[t] + np.dot(f_u[t].T, np.dot(V_xx, f_x[t]))
                Q_uu = l_uu[t] + np.dot(f_u[t].T, np.dot(V_xx, f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))

                k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(K[t].T, np.dot(Q_uu, k[t]))
                V_xx = Q_xx - np.dot(K[t].T, np.dot(Q_uu, K[t]))

            U_new = np.zeros((self.INPUT_SIZE,self.tN))
            x_new = x0.copy()
            for t in range(self.tN - 1): 
                # use feedforward (k) and feedback (K) gain matrices 
                # calculated from our value function approximation
                U_new[:,t] = U[:,t] + k[t] + np.dot(K[t], x_new - X[:,t])
                #_,x_new = self.plant_dynamics(x_new, U_new[t])

            # evaluate the new trajectory 
            X_new, cost_new = sim_n_steps(x0, U_new,xref)

            if cost_new < cost: 
                # decrease lambda (get closer to Newton's method)
                lamb /= self.lamb_factor

                X = np.copy(X_new) # update trajectory 
                U = np.copy(U_new) # update control signal
                oldcost = np.copy(cost)
                cost = np.copy(cost_new)

                sim_new_trajectory = True # do another rollout

                # check to see if update is small enough to exit
                print('...',cost[0],oldcost[0])
                if ii > 0 and ((abs(oldcost[0]-cost[0])/cost[0]) < self.eps_converge):
                    #print("Converged at iteration = %d; Cost = %.4f;"%(ii,cost_new) + 
                    #        " logLambda = %.1f"%np.log(lamb))
                    break

            else: 
                # increase lambda (get closer to gradient descent)
                lamb *= self.lamb_factor
                # print("cost: %.4f, increasing lambda to %.4f")%(cost, lamb)
                if lamb > self.lamb_max: 
                    #print("lambda > max_lambda at iteration = %d;"%ii + 
                    #    " Cost = %.4f; logLambda = %.1f"%(cost, np.log(lamb)))
                    break

        return X, U, cost
    

In [34]:
_xref=np.ones((NX,T+1),dtype=np.float64)
_xref[0,:]=0
_xref[1,:]=1
_xref[2,:]=0
_xref[3,:]=np.arctan2(3,0.0001)
_dref=np.zeros((T),dtype=np.float64)
_U=np.zeros((NU,T),dtype=np.float64)
_x0=np.zeros((NX),dtype=np.float64)
iLQR=iLQRController()
ret=iLQR.ilqr(_x0,_U,_xref)

[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849
[0. 0.] [[0.01 0.  ]
 [0.   0.01]]
124.36481908141849


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [35]:
ret[1][:,2]

array([5.14549579, 0.8703531 ])

In [36]:
ret[0][:,3]

array([8.70691834e-01, 1.07991021e-03, 3.86832534e+00, 7.06399912e-03])